In [1]:
import sys
sys.path.append('../..')
import pybeamtools.controls as pc

[INFO ] [     MainThread] [09:24:39      logging.py  23] [pybeamtools.utils.logging] Starting shared logging thread on PID 25324
[INFO ] [   log_listener] [09:24:39      logging.py  32] [pybeamtools.utils.logging] Logging listener started on PID 25324
[INFO ] [     MainThread] [09:24:39      logging.py  27] [pybeamtools.utils.logging] Logging setup finished
[INFO ] [     MainThread] [09:24:39        utils.py 145] [numexpr.utils] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO ] [     MainThread] [09:24:39        utils.py 157] [numexpr.utils] NumExpr defaulting to 8 threads.
[DEBUG] [     MainThread] [09:24:39     _backend.py  35] [caproto] Backend 'array' registered
[DEBUG] [     MainThread] [09:24:39     _backend.py  35] [caproto] Backend 'numpy' registered
[DEBUG] [     MainThread] [09:24:39     _backend.py  47] [caproto] Selecting backend: 'numpy'


### Create simulation with 3 magnets [will present in detail next week]

In [2]:
from pybeamtools.sim.core import SimulationEngine, ChannelMap, ChannelMapper
from pybeamtools.sim.devices import RealisticMagnet, Oscillator

sim = SimulationEngine()

def read_device(device, output):
    return device.read()

def write_device(device, output, value):
    return device.write(value)

mag1 = RealisticMagnet(name='mag1', value=0, low=-20.0, high=20.0, noise=0.1, resolution=None, model='instant')
m = ChannelMap(device=mag1, output='WRITE_PV_1', read_fun=read_device, write_fun=write_device)
chm = ChannelMapper(maps=[m])
sim.add_device(mag1, period=10.0)
sim.add_mapper(chm)

mag2 = RealisticMagnet(name='mag2', value=0, low=-20.0, high=20.0, noise=0.1, resolution=None, model='instant')
m = ChannelMap(device=mag2, output='READ_PV_1', read_fun=read_device)
chm = ChannelMapper(maps=[m])
sim.add_device(mag2, period=10.0)
sim.add_mapper(chm)

osc = Oscillator(name='osc', period = 20.0, amplitude = 2.0)
m = ChannelMap(device=osc, output='OSCILLATING_PV_1', read_fun=read_device)
chm = ChannelMapper(maps=[m])
sim.add_device(osc, period=5.0)
sim.add_mapper(chm)

sim.start_update_thread()
print()

[DEBUG] [     MainThread] [09:24:41         core.py 152] [SimulationEngine] Added device (mag1) with update period of (10.0)
[DEBUG] [     MainThread] [09:24:41         core.py 171] [SimulationEngine] Added mapper for outputs (['WRITE_PV_1']) from devices (['mag1'])
[DEBUG] [     MainThread] [09:24:41         core.py 152] [SimulationEngine] Added device (mag2) with update period of (10.0)
[DEBUG] [     MainThread] [09:24:41         core.py 171] [SimulationEngine] Added mapper for outputs (['READ_PV_1']) from devices (['mag2'])
[DEBUG] [     MainThread] [09:24:41         core.py 152] [SimulationEngine] Added device (osc) with update period of (5.0)
[DEBUG] [     MainThread] [09:24:41         core.py 171] [SimulationEngine] Added mapper for outputs (['OSCILLATING_PV_1']) from devices (['osc'])
[DEBUG] [     MainThread] [09:24:41         core.py 243] [SimulationEngine] Starting poll thread
[DEBUG] [sim_engine_poll] [09:24:41         core.py 198] [SimulationEngine] Hello from simulation po

In [3]:
# Simulation is ready to go
sim.channels, sim.devices

(['WRITE_PV_1', 'READ_PV_1', 'OSCILLATING_PV_1'],
 {'mag1': <pybeamtools.sim.devices.RealisticMagnet at 0x257607cf8e0>,
  'mag2': <pybeamtools.sim.devices.RealisticMagnet at 0x257607cf700>,
  'osc': <pybeamtools.sim.devices.Oscillator at 0x25747509490>})

### Configs

In [4]:
# Create options that fully define initial configuration
from pybeamtools.controls.control_lib import ConnectionOptions
ao = pc.AcceleratorOptions(connection_settings=ConnectionOptions(network='dummy'))

In [5]:
ao.dict()

{'write_settings': {},
 'readback_settings': {'delay_after_write': 1.0,
  'readback_match_timeout': 60.0,
  'delay_after_readback': 3.0,
  'total_set_and_readback_cycle_min_time': 0.0},
 'connection_settings': {'network': 'dummy', 'pvs': [], 'timeout': 5.0},
 'interlocks': []}

In [6]:
# Note that config is validated
ConnectionOptions(network='badnetwork')

ValidationError: 1 validation error for ConnectionOptions
network
  unexpected value; permitted: 'epics', 'dummy' (type=value_error.const; given=badnetwork; permitted=('epics', 'dummy'))

In [7]:
ConnectionOptions(timeout='notafloat')

ValidationError: 1 validation error for ConnectionOptions
timeout
  value is not a valid float (type=type_error.float)

### Controller

In [10]:
# Now create accelerator interface (what is used for control)
acc = pc.Accelerator(options=ao, ctx=sim)

# Add PVs
from pybeamtools.controls.network import SimPV, PVOptions, PVAccess
write_triggers = ['WRITE_PV_1']
write_pv_1 = SimPV(PVOptions(name='WRITE_PV_1', low=-5.0, high=5.0, security=PVAccess.READWRITE))
read_triggers = ['READ_PV_1']
read_pv_1 = SimPV(PVOptions(name='READ_PV_1', security=PVAccess.READONLY))
oscillating_pv_1 = SimPV(PVOptions(name='OSCILLATING_PV_1', security=PVAccess.READONLY))
pv_list = ['OSCILLATING_PV_1'] + read_triggers + write_triggers
print(f'PV NAME LIST: {pv_list}')

[INFO ] [     MainThread] [09:26:52  control_lib.py  58] [Accelerator] Control lib init
[INFO ] [     MainThread] [09:26:52      network.py 176] [SimConnectionManager] Creating dummy connection manager
[INFO ] [     MainThread] [09:26:52  control_lib.py  86] [Accelerator] Startup finished


PV NAME LIST: ['OSCILLATING_PV_1', 'READ_PV_1', 'WRITE_PV_1']


In [14]:
# Settings serialization
acc.options.dict()

{'write_settings': {},
 'readback_settings': {'delay_after_write': 1.0,
  'readback_match_timeout': 60.0,
  'delay_after_readback': 3.0,
  'total_set_and_readback_cycle_min_time': 0.0},
 'connection_settings': {'network': 'dummy',
  'pvs': [{'name': 'READ_PV_1',
    'low': None,
    'high': None,
    'monitor': True,
    'security': 1,
    'read_timeout': 2.0,
    'write_timeout': 5.0},
   {'name': 'WRITE_PV_1',
    'low': -5.0,
    'high': 5.0,
    'monitor': True,
    'security': 3,
    'read_timeout': 2.0,
    'write_timeout': 5.0},
   {'name': 'OSCILLATING_PV_1',
    'low': None,
    'high': None,
    'monitor': True,
    'security': 1,
    'read_timeout': 2.0,
    'write_timeout': 5.0}],
  'timeout': 5.0},
 'interlocks': []}

In [15]:
# Can also output as SDDS parameter list
from pybeamtools.utils.pydantic import to_sdds
to_sdds(acc.options.dict())

[('pybeamtools:readback_settings:delay_after_write', '1.0'),
 ('pybeamtools:readback_settings:readback_match_timeout', '60.0'),
 ('pybeamtools:readback_settings:delay_after_readback', '3.0'),
 ('pybeamtools:readback_settings:total_set_and_readback_cycle_min_time',
  '0.0'),
 ('pybeamtools:connection_settings:network', 'dummy'),
 ('pybeamtools:connection_settings:pvs',
  '[{"name": "READ_PV_1", "low": null, "high": null, "monitor": true, "security": 1, "read_timeout": 2.0, "write_timeout": 5.0}, {"name": "WRITE_PV_1", "low": -5.0, "high": 5.0, "monitor": true, "security": 3, "read_timeout": 2.0, "write_timeout": 5.0}, {"name": "OSCILLATING_PV_1", "low": null, "high": null, "monitor": true, "security": 1, "read_timeout": 2.0, "write_timeout": 5.0}]'),
 ('pybeamtools:connection_settings:timeout', '5.0'),
 ('pybeamtools:interlocks', '[]')]

In [13]:
# Add new PVs to controller
acc.add_pv_object([read_pv_1, write_pv_1, oscillating_pv_1])
sim.schedule_update()
print()

[DEBUG] [     MainThread] [09:27:16      network.py 212] [SimConnectionManager] Adding 3 PV objects
[DEBUG] [     MainThread] [09:27:16         core.py 184] [SimulationEngine] Created subscription for channel (READ_PV_1)
[DEBUG] [     MainThread] [09:27:16         core.py 184] [SimulationEngine] Created subscription for channel (WRITE_PV_1)
[DEBUG] [     MainThread] [09:27:16         core.py 184] [SimulationEngine] Created subscription for channel (OSCILLATING_PV_1)


In [16]:
# note how options have updated to reflect state
acc.options.dict()

{'write_settings': {},
 'readback_settings': {'delay_after_write': 1.0,
  'readback_match_timeout': 60.0,
  'delay_after_readback': 3.0,
  'total_set_and_readback_cycle_min_time': 0.0},
 'connection_settings': {'network': 'dummy',
  'pvs': [{'name': 'READ_PV_1',
    'low': None,
    'high': None,
    'monitor': True,
    'security': 1,
    'read_timeout': 2.0,
    'write_timeout': 5.0},
   {'name': 'WRITE_PV_1',
    'low': -5.0,
    'high': 5.0,
    'monitor': True,
    'security': 3,
    'read_timeout': 2.0,
    'write_timeout': 5.0},
   {'name': 'OSCILLATING_PV_1',
    'low': None,
    'high': None,
    'monitor': True,
    'security': 1,
    'read_timeout': 2.0,
    'write_timeout': 5.0}],
  'timeout': 5.0},
 'interlocks': []}

### Interacting with PVs

In [17]:
# This fails because of PV limits (no interlocks)
write_pv_1.write(5.5)
print()

InvalidWriteError: Value 5.5 above bounds (-5.0|5.0)

In [18]:
# This is allowed
write_pv_1.write(4.5)
print()

[DEBUG] [     MainThread] [09:28:16  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:28:16  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (0) of (0) interlocks
[DEBUG] [     MainThread] [09:28:16  control_lib.py 185] [Accelerator] No interlocks triggered, skip polling stage
[DEBUG] [     MainThread] [09:28:16         core.py 281] [SimulationEngine] Writing (4.5) to channel (WRITE_PV_1)


### Interlocks

In [19]:
# Add an interlock to further constraint PVs
from pybeamtools.controls.interlocks import LimitInterlock, LimitInterlockOptions
interlock_limits = {'WRITE_PV_1':(-1.0,1.0)}

lopt = LimitInterlockOptions(pv_list=pv_list,
                            read_events=[],
                            write_events=pv_list,
                            limits=interlock_limits)
ilock = LimitInterlock(options=lopt)
acc.add_interlock(ilock)
print()

[DEBUG] [     MainThread] [09:29:40  distributed.py  99] [ProcessManager] Interlock worker started on PID 21864
[INFO ] [     MainThread] [09:29:40  control_lib.py 247] [Accelerator] Interlock (4d6bb552-2a6e-4049-9140-5896d70c6980) with PV list (['OSCILLATING_PV_1', 'READ_PV_1', 'WRITE_PV_1']) added


[INFO ] [     MainThread] [09:29:41      logging.py  56] [pybeamtools.utils.logging] Starting worker logging finished


In [20]:
# Same write now fails due to interlocks
write_pv_1.write(4.5)
print()

[DEBUG] [     MainThread] [09:29:57  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:29:57  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (1) of (1) interlocks
[DEBUG] [     MainThread] [09:29:57  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3])
[DEBUG] [     MainThread] [09:29:57  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 4.5}, 'data': {'WRITE_PV_1': array([4.51104597]), 'READ_PV_1': array([0.00217953]), 'OSCILLATING_PV_1': array([-1.99210341])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:29:57  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 4.5}, 'data': {'WRITE_PV_1': array([4.51104597]), 'READ_PV_1': array([0.00217953]), 'OSCILLATING_PV_1': array([-1.99210341])}, 'timestamps': None} (id 698158) to interlock (4d6bb552-2a6e-4049-9140-5896d70c6980)
[INFO ] [     MainThread] [09:29:57  distribu

InterlockWriteError: Write on (['WRITE_PV_1']) of ([4.5]) failed due to interlock

In [21]:
# Smaller write is allowed
write_pv_1.write(0.5)
print()

[DEBUG] [     MainThread] [09:30:26  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:30:26  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (1) of (1) interlocks
[DEBUG] [     MainThread] [09:30:26  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3])
[DEBUG] [     MainThread] [09:30:26  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([4.5781034]), 'READ_PV_1': array([0.04285462]), 'OSCILLATING_PV_1': array([0.17829916])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:30:26  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([4.5781034]), 'READ_PV_1': array([0.04285462]), 'OSCILLATING_PV_1': array([0.17829916])}, 'timestamps': None} (id 137549) to interlock (4d6bb552-2a6e-4049-9140-5896d70c6980)
[INFO ] [     MainThread] [09:30:26  distributed.

In [23]:
# Another interlock to block all writes when oscillator PV < 0
lopt2 = LimitInterlockOptions(pv_list=pv_list,
                            read_events=[],
                            write_events=write_triggers,
                            limits={'OSCILLATING_PV_1':(-10.0,0.0)},
                            block_all_writes=True)
ilock2 = LimitInterlock(options=lopt2)
acc.add_interlock(ilock2)

[DEBUG] [     MainThread] [09:31:15  distributed.py  99] [ProcessManager] Interlock worker started on PID 22552
[INFO ] [     MainThread] [09:31:15  control_lib.py 247] [Accelerator] Interlock (9bfe7bac-37f2-426a-934f-4d0447be4d94) with PV list (['OSCILLATING_PV_1', 'READ_PV_1', 'WRITE_PV_1']) added


[INFO ] [     MainThread] [09:31:17      logging.py  56] [pybeamtools.utils.logging] Starting worker logging finished


In [25]:
while True:
    import time
    write_pv_1.write(0.5)
    time.sleep(2)
    print()

[DEBUG] [     MainThread] [09:31:58  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:31:58  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [09:31:58  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [09:31:58  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04424846]), 'OSCILLATING_PV_1': array([-1.99178463])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04424846]), 'OSCILLATING_PV_1': array([-1.99178463])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:31:58  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04

[DEBUG] [     MainThread] [09:32:00  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:32:00  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [09:32:00  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [09:32:00  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04424846]), 'OSCILLATING_PV_1': array([-1.99178463])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04424846]), 'OSCILLATING_PV_1': array([-1.99178463])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:32:00  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04

[DEBUG] [     MainThread] [09:32:02  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:32:02  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [09:32:02  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [09:32:02  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04424846]), 'OSCILLATING_PV_1': array([0.17733557])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.04424846]), 'OSCILLATING_PV_1': array([0.17733557])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:32:02  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.39176043]), 'READ_PV_1': array([0.0442

InterlockWriteError: Write on (['WRITE_PV_1']) of ([0.5]) failed due to interlock

In [26]:
# Finally, let's limit maximum setting rate to 2Hz
acc.remove_interlock(ilock2)

from pybeamtools.controls.interlocks import RatelimitInterlock,RatelimitInterlockOptions
lopt3 = RatelimitInterlockOptions(pv_list=pv_list,
                            read_events=[],
                            write_events=write_triggers,
                            min_delay=0.5)
ilock3 = RatelimitInterlock(options=lopt3)
acc.add_interlock(ilock3)

[DEBUG] [     MainThread] [09:33:01  distributed.py  90] [ProcessManager] Terminating worker process_read 22552
[INFO ] [     MainThread] [09:33:01  control_lib.py 253] [Accelerator] Interlock (9bfe7bac-37f2-426a-934f-4d0447be4d94) with PV list (['OSCILLATING_PV_1', 'READ_PV_1', 'WRITE_PV_1']) removed
[DEBUG] [     MainThread] [09:33:01  distributed.py  99] [ProcessManager] Interlock worker started on PID 8620
[INFO ] [     MainThread] [09:33:01  control_lib.py 247] [Accelerator] Interlock (2670c2d8-21e2-41c9-9996-03c5fe78bc2b) with PV list (['OSCILLATING_PV_1', 'READ_PV_1', 'WRITE_PV_1']) added


[INFO ] [     MainThread] [09:33:02      logging.py  56] [pybeamtools.utils.logging] Starting worker logging finished


In [28]:
# This is below rate limit and works
for i in range(5):
    time.sleep(1)
    write_pv_1.write(0.5)

[DEBUG] [     MainThread] [09:33:20  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:33:20  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [09:33:20  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [09:33:20  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.29100247]), 'READ_PV_1': array([-0.31402651]), 'OSCILLATING_PV_1': array([-1.99169308])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.29100247]), 'READ_PV_1': array([-0.31402651]), 'OSCILLATING_PV_1': array([-1.99169308])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:33:20  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.29100247]), 'READ_PV_1': array([-0

In [29]:
# This is too fast
for i in range(5):
    time.sleep(0.4)
    write_pv_1.write(0.5)

[DEBUG] [     MainThread] [09:33:30  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:33:30  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [09:33:30  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [09:33:30  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.33816369]), 'READ_PV_1': array([0.02712348]), 'OSCILLATING_PV_1': array([1.99200668])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.33816369]), 'READ_PV_1': array([0.02712348]), 'OSCILLATING_PV_1': array([1.99200668])}, 'timestamps': None}]
[DEBUG] [     MainThread] [09:33:30  distributed.py 127] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.33816369]), 'READ_PV_1': array([0.0271

InterlockWriteError: Write on (['WRITE_PV_1']) of ([0.5]) failed due to interlock

### Fail-safe handling of code crashes

In [30]:
acc.interlocks

In [31]:
# Each interlock runs in its own process (in future, can be on another node)
process = acc.pm.process_map[ilock3.uuid]
process, process.pid

(<SpawnProcess name='SpawnProcess-3' pid=8620 parent=25324 started>, 8620)

In [32]:
# We fake a crash by killing process with a syscall
import psutil
p = psutil.Process(process.pid)
print(p)
p.terminate()
time.sleep(1.0)
print(p)

psutil.Process(pid=8620, name='python.exe', status='running', started='09:33:01')
psutil.Process(pid=8620, name='python.exe', status='terminated', started='09:33:01')


In [33]:
write_pv_1.write(0.5)

[DEBUG] [     MainThread] [09:34:46  control_lib.py 179] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [09:34:46  control_lib.py 182] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [09:34:46  control_lib.py 206] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [09:34:46  control_lib.py 207] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.48822604]), 'READ_PV_1': array([-0.03868458]), 'OSCILLATING_PV_1': array([0.17467465])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'WRITE_PV_1': array([0.48822604]), 'READ_PV_1': array([-0.03868458]), 'OSCILLATING_PV_1': array([0.17467465])}, 'timestamps': None}]


ControlLibException: Interlock process_read terminated unexpectedly with code (15)

### Settings save/restore

In [ ]:
options_dump = acc.options.dict()
options_dump

In [ ]:
acc.options.json()

In [ ]:
import json
pc.AcceleratorOptions.parse_obj(json.loads(acc.options.json()))

In [ ]:
acc_restored_options = pc.AcceleratorOptions.parse_obj(options_dump)
acc_restored_options

In [ ]:
acc_new = pc.Accelerator(acc_restored_options, ctx=sim)